## 2. LDA


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from functools import reduce
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
blood_pressure_path= "/Users/yuxhe/Desktop/master/242final/BPX_I.XPT"
demographic_path= "/Users/yuxhe/Desktop/master/242final/DEMO_I.XPT"
dietary_path = "/Users/yuxhe/Desktop/master/242final/DR1TOT_I.XPT"
body_measures_path = "/Users/yuxhe/Desktop/master/242final/BMX_I.XPT"
blood_pressure = pd.read_sas(blood_pressure_path)
demographic = pd.read_sas(demographic_path)
dietary = pd.read_sas(dietary_path)
body_measures = pd.read_sas(body_measures_path)

In [ ]:
# Dependent Variables
patient='SEQN'
age = 'RIDAGEYR'
gender = 'RIAGENDR'
systolic_2nd_reading='BPXSY2'
diastolic_2nd_reading = 'BPXDI2'
systolic_3rd_reading='BPXSY3'
diastolic_3rd_reading = 'BPXDI3'
calcium = 'DR1TCALC'
phosphore = 'DR1TPHOS'
magnesium = 'DR1TMAGN'
iron = 'DR1TIRON'
zinc = 'DR1TZINC'
copper = 'DR1TCOPP'
sodium = 'DR1TSODI'
potassium = 'DR1TPOTA'
selenium = 'DR1TSELE'
sugar = 'DR1TSUGR'
caffeine = 'DR1TCAFF'
fat = 'DR1TTFAT'
weight = 'BMXWT'
height = 'BMXHT'
bmi = 'BMI'
# activity = 'PAQ620'
race = 'RIDRETH1'
income = 'INDHHIN2'

In [ ]:
# Create a merged data frame of patients characteristics
cm_blood_pressure = blood_pressure.drop(blood_pressure.columns.difference([patient,systolic_2nd_reading, diastolic_2nd_reading, systolic_3rd_reading, diastolic_3rd_reading]), axis=1, inplace=False)
cm_demographic = demographic.drop(demographic.columns.difference([patient, age, gender, race, income]), axis=1, inplace=False)
cm_dietary = dietary.drop(dietary.columns.difference([patient, calcium, phosphore, magnesium, iron, zinc, copper, sodium, potassium, selenium, sugar, caffeine, fat]), axis=1, inplace=False)
cm_body_measures = body_measures.drop(body_measures.columns.difference([patient, weight, height]), axis=1, inplace=False)

In [ ]:
#Remove NaN values
cm_blood_pressure.dropna(inplace=True)
cm_dietary.dropna(inplace=True)
cm_body_measures.dropna(inplace=True)
cm_demographic.dropna(inplace=True)
cm_demographic = cm_demographic[cm_demographic[income] != 12]
cm_demographic = cm_demographic[cm_demographic[income] != 77]
cm_demographic = cm_demographic[cm_demographic[income] != 99]
print(cm_blood_pressure)
print(cm_dietary)
print(cm_body_measures)

         SEQN  BPXSY2  BPXDI2  BPXSY3  BPXDI3
0     83732.0   124.0    64.0   116.0    62.0
1     83733.0   140.0    88.0   134.0    82.0
2     83734.0   132.0    44.0   136.0    46.0
3     83735.0   134.0    68.0   136.0    70.0
4     83736.0   114.0    54.0    98.0    56.0
...       ...     ...     ...     ...     ...
9537  93696.0   116.0    76.0   114.0    72.0
9538  93697.0   146.0    58.0   144.0    52.0
9541  93700.0   106.0    66.0   104.0    68.0
9542  93701.0   114.0    46.0   114.0    52.0
9543  93702.0   114.0    68.0   124.0    64.0

[7231 rows x 5 columns]
         SEQN  DR1TSUGR  DR1TTFAT  DR1TCALC  DR1TPHOS  DR1TMAGN  DR1TIRON  \
0     83732.0     42.31     79.24     623.0    1052.0     255.0     16.01   
1     83733.0    180.84     77.91     594.0    1414.0     262.0     11.01   
2     83734.0     62.87     91.97     872.0    1527.0     497.0     26.17   
3     83735.0     54.77     49.23    1284.0    1439.0     318.0      9.07   
4     83736.0     71.84     19.63     

In [ ]:
# Add a new column 'systolic_bp' with the average of 'systolic_1st_reading' and 'systolic_2nd_reading'
cm_blood_pressure['systolic_bp'] = cm_blood_pressure[[systolic_2nd_reading, systolic_3rd_reading]].mean(axis=1)
cm_blood_pressure['diastolic_bp'] = cm_blood_pressure[[diastolic_2nd_reading, diastolic_3rd_reading]].mean(axis=1)
cm_blood_pressure = cm_blood_pressure.drop([systolic_2nd_reading, systolic_3rd_reading], axis=1)
cm_blood_pressure = cm_blood_pressure.drop([diastolic_2nd_reading, diastolic_3rd_reading], axis=1)

In [ ]:
def categorize_hypertension(row):
    if row['systolic_bp'] < 120 and row['diastolic_bp'] < 80:
        return 0 # normal
    elif row['systolic_bp'] < 129 and row['diastolic_bp'] < 80:
        return 1 # elevated
    elif row['systolic_bp'] < 139 and row['diastolic_bp'] < 89:
        return 2 # stage 1
    else:
        return 3 # stage 2

def categorize_income(row):
    if row[income] == 13:
        return 'Poor'
    elif row[income] <= 4 :
        return 'Poor'
    elif row[income] >4 and row[income]<10:
        return 'Middle'
    elif row[income] >= 10:
        return 'Rich'
cm_blood_pressure['hypertension'] = cm_blood_pressure.apply(categorize_hypertension,axis=1)
cm_demographic['income_category'] = cm_demographic.apply(categorize_income, axis=1)

In [ ]:
dataframes = [cm_blood_pressure, cm_dietary, cm_demographic, cm_body_measures]
merged_data = reduce(lambda left, right: pd.merge(left, right, on='SEQN'),dataframes)
merged_data.drop(patient,axis=1,inplace=True)
merged_data[bmi] = merged_data[weight] / ((0.01)*merged_data[height])**2
data_male = merged_data[merged_data[gender] == 1]
data_female = merged_data[merged_data[gender] == 2]
print(merged_data)
print(len(merged_data[merged_data['hypertension'] == 0]))
print(len(merged_data[merged_data['hypertension'] == 1]))
print(len(merged_data[merged_data['hypertension'] == 2]))
print(len(merged_data[merged_data['hypertension'] == 3]))

      systolic_bp  diastolic_bp  hypertension  DR1TSUGR  DR1TTFAT  DR1TCALC  \
0           120.0          63.0             1     42.31     79.24     623.0   
1           137.0          85.0             2    180.84     77.91     594.0   
2           134.0          45.0             2     62.87     91.97     872.0   
3           135.0          69.0             2     54.77     49.23    1284.0   
4           106.0          55.0             0     71.84     19.63      72.0   
...           ...           ...           ...       ...       ...       ...   
6051        111.0          47.0             0     64.07     87.47     646.0   
6052        115.0          74.0             0     40.70     82.23     610.0   
6053        145.0          55.0             3     52.88     60.05     883.0   
6054        114.0          49.0             0    160.79    105.57    1861.0   
6055        119.0          66.0             0     42.70     42.88     525.0   

      DR1TPHOS  DR1TMAGN  DR1TIRON  DR1TZINC  ...  

In [ ]:
X = merged_data.drop(['hypertension','diastolic_bp','systolic_bp'], axis=1)
y = merged_data['hypertension']

In [ ]:
print(merged_data.dtypes)

systolic_bp        float64
diastolic_bp       float64
hypertension         int64
DR1TSUGR           float64
DR1TTFAT           float64
DR1TCALC           float64
DR1TPHOS           float64
DR1TMAGN           float64
DR1TIRON           float64
DR1TZINC           float64
DR1TCOPP           float64
DR1TSODI           float64
DR1TPOTA           float64
DR1TSELE           float64
DR1TCAFF           float64
RIAGENDR           float64
RIDAGEYR           float64
RIDRETH1           float64
INDHHIN2           float64
income_category     object
BMXWT              float64
BMXHT              float64
BMI                float64
dtype: object


In [ ]:
merged_data = pd.get_dummies(merged_data, columns=['income_category'])

In [ ]:
scaler = StandardScaler()
X = merged_data.drop(['hypertension','diastolic_bp','systolic_bp'], axis=1)  # Ensure 'hypertension' is dropped after encoding
X_scaled = scaler.fit_transform(X)

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, merged_data['hypertension'], test_size=0.3, random_state=242)

# Applying LDA
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.5877820583379196
Confusion Matrix:
 [[879   7  31  72]
 [174   3  21  63]
 [172   8  37  81]
 [ 85   2  33 149]]


In [ ]:
# try bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score
lda = LinearDiscriminantAnalysis()

# Bagging classifier with LDA
bagging = BaggingClassifier(lda, n_estimators=10, random_state=42)
bagging.fit(X_train, y_train)

# Predict and evaluate
y_pred = bagging.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy with Bagging LDA: {accuracy}")

Accuracy with Bagging LDA: 0.5855806274078151


In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, classification_report

# Create a Bagging ensemble of LDA models
bagging_lda = BaggingClassifier(
    base_estimator=LinearDiscriminantAnalysis(),
    n_estimators=10,  # Number of different LDA models to train
    random_state=42,
    max_samples=0.8,  # Percentage of data to sample for training each LDA model
    max_features=1.0  # Percentage of features to sample for each LDA model
)

# Train the Bagging ensemble
bagging_lda.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=LinearDiscriminantAnalysis(), max_samples=0.8,
                  random_state=42)

In [ ]:
# Predict on the test set
y_pred = bagging_lda.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5844799119427628


In [ ]:
# k-fold

In [ ]:
from sklearn.model_selection import KFold, cross_val_score

In [ ]:
# Set up K-Fold
k = 5
kf = KFold(n_splits=k, shuffle=True, random_state=242)
lda = LinearDiscriminantAnalysis()

In [ ]:
# Evaluate the model
scores = cross_val_score(lda, X_train, y_train, cv=kf, scoring='accuracy')
print("Accuracy scores for each fold:")
print(scores)
print(f"Mean accuracy across all folds: {np.mean(scores):.2f} ± {np.std(scores):.2f}")

Accuracy scores for each fold:
[0.57311321 0.57900943 0.59787736 0.55660377 0.57615112]
Mean accuracy across all folds: 0.58 ± 0.01
